https://www.analyticsvidhya.com/blog/2016/06/quick-guide-build-recommendation-engine-python/

In [ ]:
import pandas as pd

In [2]:
# pass in column names for each CSV and read them using pandas. 
# Column names available in the readme file

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('data/ml-100k/u.user', sep='|', names=u_cols,
 encoding='latin-1')
print(users.shape)
users.head()

(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [3]:
#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('data/ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')
print(ratings.shape)
ratings.head()

(100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('data/ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')
print(items.shape)
items.head()

(1682, 24)


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
#divide the ratings data set into test and train data for making models
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('data/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('data/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
print(ratings_base.shape, ratings_test.shape)

((90570, 4), (9430, 4))


In [6]:
#using GraphLab, lets convert these to SFrames
import graphlab
train_data = graphlab.SFrame(ratings_base)
test_data = graphlab.SFrame(ratings_test)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\DEXTER~1.DCR\AppData\Local\Temp\graphlab_server_1556070211.log.0


This non-commercial license of GraphLab Create for academic use is assigned to dexter.dcruz@gmail.com and will expire on April 13, 2020.


### Lets start with making a popularity based model,

* train_data: the SFrame which contains the required data
* user_id: the column name which represents each user ID
* item_id: the column name which represents each item to be recommended
* target: the column name representing scores/ratings given by the user

In [7]:
# Lets start with making a popularity based model, we can train a recommendation as:
popularity_model = graphlab.popularity_recommender\
    .create(train_data, user_id='user_id', item_id='movie_id', target='rating')

Recsys training: model = popularity

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.081595s

90570 observations to process; with 1680 unique items.

In [8]:
#Get recommendations for first 5 users and print them
#users = range(1,6) specifies user ID of first 5 users
#k=5 specifies top 5 recommendations to be given
popularity_recomm = popularity_model.recommend(users=range(1,6),k=5)
# popularity_recomm.print_rows(num_rows=25)
popularity_recomm.show()

Canvas is accessible via web browser at the URL: http://localhost:63698/index.html
Opening Canvas in default web browser.


In [9]:
# List of popular movies
items[items['movie id'].isin([1536, 1201, 1189, 1122, 814])]

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
813,814,"Great Day in Harlem, A (1994)",01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Great%20Day%2...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1121,1122,They Made Me a Criminal (1939),01-Jan-1939,NaN,http://us.imdb.com/M/title-exact?They%20Made%2...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1188,1189,Prefontaine (1997),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Prefontaine%2...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1200,1201,Marlene Dietrich: Shadow and Light (1996),02-Apr-1996,NaN,http://us.imdb.com/M/title-exact?Marlene%20Die...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1535,1536,Aiqing wansui (1994),22-Jul-1996,NaN,http://us.imdb.com/M/title-exact?Aiqing%20Wans...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#Lets create a model based on item similarity using pearson as follows:
#Train Model
item_sim_model_pearson = graphlab.item_similarity_recommender\
    .create(train_data, user_id='user_id', item_id='movie_id', target='rating', similarity_type='pearson')
#Make Recommendations:
item_sim_recomm_pearson = item_sim_model_pearson.recommend(users=range(1,6),k=5)
item_sim_recomm_pearson.print_rows(num_rows=25)
item_sim_recomm_pearson.show()

Recsys training: model = item_similarity

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.353533s

+---------+----------+-------+------+
| user_id | movie_id | score | rank |
+---------+----------+-------+------+
|    1    |   1293   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    2    |   1293   |  5.0  |  1   |
|    2    |   1201   |  5.0  |  2   |
|    2    |   1189   |  5.0  |  3   |
|    2    |   1122   |  5.0  |  4   |
|    2    |   814    |  5.0  |  5   |
|    3    |   1293   |  5.0  |  1   |
|    3    |   1201   |  5.0  |  2   |
|    3    |   1189   |  5.0  |  3   |
|    3    |   1122   |  5.0  |  4   |
|    3    |   814    |  5.0  |  5   |
|    4    |   1293   |  5.0  |  1   |
|    4    |   1201   |  5.0  |  2   |
|    4    |   1189   |  5.0  |  3   |
|    4    |   1122   |  5.0  |  4   |
|    4    |   814    |  5.0  |  5   |
|    5    |   1293   |  5.0  |  1   |
|    5    |   1201   |  5.0  |  2   |
|    5    |   1189   |  5.0  |  3   |
|    5    | 

In [12]:
#Lets create a model based on item similarity as follows:
#Train Model
item_sim_model_cosine = graphlab.item_similarity_recommender\
    .create(train_data, user_id='user_id', item_id='movie_id', target='rating', similarity_type='cosine')

#Make Recommendations:
item_sim_recomm_cosine = item_sim_model_cosine.recommend(users=range(1,6),k=5)
item_sim_recomm_cosine.print_rows(num_rows=25)
item_sim_recomm_cosine.show()

Recsys training: model = item_similarity

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

+---------+----------+----------------+------+
| user_id | movie_id |     score      | rank |
+---------+----------+----------------+------+
|    1    |   423    | 0.986522585836 |  1   |
|    1    |   202    | 0.943134688012 |  2   |
|    1    |   655    | 0.791719268069 |  3   |
|    1    |   403    | 0.765623665038 |  4   |
|    1    |   568    | 0.751179529283 |  5   |
|    2    |    50    | 1.12562584877  |  1   |
|    2    |   181    | 1.06517731685  |  2   |
|    2    |    7     | 0.975477427244 |  3   |
|    2    |   121    | 0.922513024165 |  4   |
|    2    |    9     | 0.831989913033 |  5   |
|    3    |   313    | 0.635376662016 |  1   |
|    3    |   328    | 0.603288030083 |  2   |
|    3    |   315    | 0.542258712378 |  3   |
|    3    |   331    | 0.535507185893 |  4   |
|    3    |   332    | 0.531669611281 |  5   |
|    4    |    50    | 1.13114770821  |  1   |
|    4    |   288    | 1.04871511459  |  2   |
|    4    |   181    | 0.950599938631 |  3   |
|    4    |  

Finished training in 0.115543s

Canvas is updated and available in a tab in the default browser.


In [13]:
# List of movies recommended for user_id = 1 based on item similiarty with cosine
items[items['movie id'].isin([423, 202, 655, 403, 568])]

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
201,202,Groundhog Day (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Groundhog%20D...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
402,403,Batman (1989),01-Jan-1989,NaN,http://us.imdb.com/M/title-exact?Batman%20(1989),0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
422,423,E.T. the Extra-Terrestrial (1982),01-Jan-1982,NaN,http://us.imdb.com/M/title-exact?E%2ET%2E%20th...,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
567,568,Speed (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Speed%20(1994/I),0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
654,655,Stand by Me (1986),01-Jan-1986,NaN,http://us.imdb.com/M/title-exact?Stand%20by%20...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Lets create a model based on item similarity as follows:
#Train Model
item_sim_model_jaccard = graphlab.item_similarity_recommender\
    .create(train_data, user_id='user_id', item_id='movie_id', target='rating', similarity_type='jaccard')
#Make Recommendations:
item_sim_recomm_jaccard = item_sim_model_jaccard.recommend(users=range(1,6),k=5)
item_sim_recomm_jaccard.print_rows(num_rows=25)
item_sim_recomm_jaccard.show()

Recsys training: model = item_similarity

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Canvas is updated and available in a tab in the default browser.


Finished training in 0.082026s

In [16]:
#Lets compare both the models we have built till now based on precision-recall characteristics
model_performance = graphlab.compare(test_data, [popularity_model, item_sim_model_pearson])
graphlab.show_comparison(model_performance,[popularity_model, item_sim_model_pearson])
# model_performance = graphlab.compare(test_data, [item_sim_model_jaccard, item_sim_model_cosine])
# graphlab.show_comparison(model_performance,[item_sim_model_jaccard, item_sim_model_cosine])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    |        0.0        |        0.0        |
|   3    |        0.0        |        0.0        |
|   4    | 0.000265111346766 | 0.000106044538706 |
|   5    | 0.000212089077413 | 0.000106044538706 |
|   6    | 0.000176740897844 | 0.000106044538706 |
|   7    | 0.000151492198152 | 0.000106044538706 |
|   8    | 0.000132555673383 | 0.000106044538706 |
|   9    | 0.000235654530458 | 0.000212089077413 |
|   10   | 0.000212089077413 | 0.000212089077413 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--

In [15]:
#Lets compare both the models we have built till now based on precision-recall characteristics
model_performance = graphlab.compare(test_data, [item_sim_model_jaccard, item_sim_model_cosine])
graphlab.show_comparison(model_performance,[item_sim_model_jaccard, item_sim_model_cosine])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+----------------+-----------------+
|   1    | 0.331919406151 | 0.0331919406151 |
|   2    | 0.301166489926 | 0.0602332979852 |
|   3    | 0.275715800636 | 0.0827147401909 |
|   4    | 0.253181336161 |  0.101272534464 |
|   5    | 0.234994697773 |  0.117497348887 |
|   6    | 0.220395899611 |  0.132237539767 |
|   7    | 0.209665202242 |  0.146765641569 |
|   8    | 0.200026511135 |  0.160021208908 |
|   9    | 0.190526687876 |  0.171474019088 |
|   10   | 0.183987274655 |  0.183987274655 |
+--------+----------------+-----------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+----------------+-----------------+
|   1    | 0.394485683987 | 0.0394485

## lets use the auto model picker for this dataset

In [ ]:
import graphlab as gl
# train_data, user_id='user_id', item_id='movie_id', target='rating'
model = gl.classifier.create(train_data, target='rating',
                             features = ['user_id',
                                         'movie_id'])


## lets make some predictions based on this recommended model

In [ ]:
predictions = model.classify(test_data)

##  Evaluate the model, with the results stored in a dictionary

In [ ]:
results = model.evaluate(test_data)

In [ ]:
results

In [ ]:

# List movies rated by user_id = 1
# items[items['movie id'].isin([423, 568, 202, 403, 385])]

# List of popular movies
items[items['movie id'].isin([1500, 1201, 1189, 1122, 814])]
# items.head()